In [1]:
# Code for task 2
!pip install qiskit
import qiskit
from qiskit import  transpile
from qiskit.providers.basic_provider import BasicProvider
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

'''
from qiskit import transpile
    provider = BasicProvider()
    backend = provider.get_backend("basic_simulator")
    job = backend.run(qc)
    counts = job.result().get_counts(qc)

 Need to use the above for the new version of qiskit (1 or above) in place of the old execute() function and "qasm_simulator"
'''


'\nfrom qiskit import transpile\n    provider = BasicProvider()\n    backend = provider.get_backend("basic_simulator")\n    job = backend.run(qc)\n    counts = job.result().get_counts(qc)\n\n  provider = BasicProvider()\n  backend = provider.get_backend("basic_simulator")\n  job = backend.run(circuit)\n  counts = job.result().get_counts(circuit)\n'

In [27]:
# Creating two functions that result in what we need.
def create_odd_to_even_circuit(lst, n):
    num_qubits = len(bin(n - 1)) - 2  # Number of qubits needed to represent numbers up to n (given in problem)
    qreg = QuantumRegister(num_qubits, name='q')
    ancilla = QuantumRegister(num_qubits, name='anc')
    creg = ClassicalRegister(num_qubits, name='c')
    qc = QuantumCircuit(qreg, ancilla, creg)

    # Apply Hadamard gates to all qubits
    qc.h(qreg)

    # Apply modular addition gate for each odd number in the list
    for num in lst:
        if num % 2 == 1:
            bin_num = bin(num)[2:].zfill(num_qubits)
            for j, bit in enumerate(reversed(bin_num)):
                if bit == '1':
                    qc.cx(qreg[num_qubits - j - 1], ancilla[j])

    # Apply controlled-X gates to update the original list
    for i in range(len(ancilla)):
        qc.cx(ancilla[i], qreg[i])

    # Measure the qubits to collapse the superposition (from the Hadamard gates)
    qc.measure(qreg, creg)

    return qc

# Main function, using the quantum circuit developed in the previous function.

def odd_to_even(lst, n):
    qc = create_odd_to_even_circuit(lst, n) # Getting circuit from other function
    # Simulating/running results
    provider = BasicProvider()
    backend = provider.get_backend("basic_simulator")
    job = backend.run(qc)
    counts = job.result().get_counts(qc)

    # Converting the measured counts to decimal and update the original list
    for binary_str, count in counts.items():
        decimal_val = int(binary_str, 2)
        for i, num in enumerate(lst):
            if num % 2 == 1:
                even_num = (num + decimal_val) % n  # Apply modular arithmetic to stay within the range [1, n); used to guarantee optimal results
                lst[i] = even_num if even_num != 0 else n  # Ensure even_num is not 0 (mod n)

    return lst

# Example usage (same as given, all results are below 31 [n] and are all even)
n = 31
lst = [1, 2, 2, 4, 5, 6, 7, 11, 17, 21, 22, 23]
result = odd_to_even(lst, n)
print(result)

[22, 2, 2, 4, 26, 6, 28, 11, 17, 4, 22, 6]
